<a href="https://colab.research.google.com/github/Abubakrhegazi/Arabic-Grammar-AutoCorrection/blob/main/Arabic-GED-GEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# prompt: import from drive folder named grammar and read an m2 file

from google.colab import drive
drive.mount('/content/drive')

# file_path = '/content/drive/My Drive/grammar/QALB-Train2014.m2'



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/grammar/QALB-Train2014.m2'

In [12]:
#functions dictionary
def smart_open(fname, mode = 'r'):
    if fname.endswith('.gz'):
        import gzip
        # Using max compression (9) by default seems to be slow.
        # Let's try using the fastest.
        return gzip.open(fname, mode, 1)
    else:
        return open(fname, mode)

def load_annotation(gold_file):
    source_sentences = []
    gold_edits = []
    fgold = smart_open(gold_file, 'r')
    puffer = fgold.read()
    fgold.close()
    #puffer = puffer.decode('utf8')
    for item in paragraphs(puffer.splitlines(True)):
        item = item.splitlines(False)
        sentence = [line[2:].strip() for line in item if line.startswith('S ')]
        assert sentence != []
        annotations = {}
        for line in item[1:]:
            if line.startswith('I ') or line.startswith('S '):
                continue
            assert line.startswith('A ')
            line = line[2:]
            fields = line.split('|||')
            #print(fields)
            start_offset = int(fields[0].split()[0])
            end_offset = int(fields[0].split()[1])
            etype = fields[1]
            if etype == 'noop':
                start_offset = -1
                end_offset = -1
            corrections =  [c.strip() if c != '-NONE-' else '' for c in fields[2].split('||')]
            # NOTE: start and end are *token* offsets
            original = ' '.join(' '.join(sentence).split()[start_offset:end_offset])
            if len(fields) > 4:
              annotator = int(fields[5])
            if annotator not in annotations.keys():
                annotations[annotator] = []
            annotations[annotator].append((start_offset, end_offset, original, corrections))
        tok_offset = 0
        for this_sentence in sentence:
            tok_offset += len(this_sentence.split())
            source_sentences.append(this_sentence)
            this_edits = {}
            for annotator, annotation in annotations.items():
                this_edits[annotator] = [edit for edit in annotation if edit[0] <= tok_offset and edit[1] <= tok_offset and edit[0] >= 0 and edit[1] >= 0]
            if len(this_edits) == 0:
                this_edits[0] = []
            gold_edits.append(this_edits)
    return (source_sentences, gold_edits)

def paragraphs(lines, is_separator=lambda x : x == '\n', joiner=''.join):
    paragraph = []
    for line in lines:
        if is_separator(line):
            if paragraph:
                yield joiner(paragraph)
                paragraph = []
        else:
            paragraph.append(line)
    if paragraph:
        yield joiner(paragraph)

In [13]:
m2_list_sentence=[]
m2_list_operation=[]

m2_list_sentence.append(load_annotation('/content/drive/MyDrive/Grammar/QALB-Train2014.m2')[0])
m2_list_operation.append(load_annotation('/content/drive/MyDrive/Grammar/QALB-TEst2014.m2')[1])


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Grammar/QALB-TEst2014.m2'

In [ ]:
sentences = []
corrections = []
current_sentence = ""
current_edits = []

for line in lines:
    line = line.strip()
    if line.startswith("S "):
        if current_sentence:
            sentences.append(current_sentence)
            corrections.append(current_edits)
        current_sentence = line[2:]
        current_edits = []
    elif line.startswith("A "):
        current_edits.append(line[2:])
# Add last entry
if current_sentence:
    sentences.append(current_sentence)
    corrections.append(current_edits)
